In [0]:
!wget https://raw.githubusercontent.com/yao8839836/text_gcn/master/data/mr/text_train.txt
!wget https://raw.githubusercontent.com/yao8839836/text_gcn/master/data/mr/label_train.txt
!wget https://raw.githubusercontent.com/yao8839836/text_gcn/master/data/mr/text_test.txt
!wget https://raw.githubusercontent.com/yao8839836/text_gcn/master/data/mr/label_test.txt

In [0]:
import os
import re
import sys
import pickle
import time
import copy
from copy import deepcopy
import numpy as np
import pandas as pd
import sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import gensim
from gensim.models import doc2vec,fasttext,FastText
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [3]:
import tensorflow as tf
import keras
from keras.layers import *
from keras.models import Model,Sequential
from keras.callbacks import *
from keras.metrics import *
import keras.backend as K
from keras.engine import Layer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [0]:
def preprocess(string):
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.lower().strip()

In [0]:
traincorpus = []
with open('text_train.txt','r',encoding='latin1') as f:
    for line in f.readlines():
        traincorpus.append(preprocess(line[:-1]))

trainlabels = []
with open('label_train.txt','r',encoding='latin1') as f:
    for line in f.readlines():
        trainlabels.append(line[:-1])

trainlabels = np.asarray(trainlabels,'int32')
traincorpus,trainlabels = shuffle(traincorpus,trainlabels,random_state=0)
Xtrain,Xval,ytrain,yval = train_test_split(traincorpus,trainlabels,test_size=0.2,random_state=42)

In [0]:
Xtest = []
with open('text_test.txt','r',encoding='latin1') as f:
    for line in f.readlines():
        Xtest.append(preprocess(line[:-1]))

ytest = []
with open('label_test.txt','r',encoding='latin1') as f:
    for line in f.readlines():
        ytest.append(line[:-1])

ytest = np.asarray(ytest,'int32')

In [0]:
train_l = len(Xtrain)
val_l = len(Xval)
test_l = len(Xtest)

In [0]:
words = []
for sentence in Xtrain:
    words.append(sentence.split())

In [0]:
batchsize = 50
numepochs = 30

## FastText + LSTM

In [0]:
mincount = 1
vecsize = 100
winsize = 5
totalepoch = 20

In [0]:
np.random.seed(0)
ft = FastText(min_count=mincount,size=vecsize,window=winsize)
ft.build_vocab(sentences=words)
ft.train(sentences=words,total_examples=len(words),epochs=totalepoch)

In [0]:
def load_vectors(sentence):
    senlen = len(sentence.split())
    maxlen = min(maxsenlen,senlen)
    vec = [ft.wv[sentence[i]] for i  in range(maxlen) if sentence[i] in ft.wv.vocab]
    if(len(vec)<maxsenlen):
        vec+=[np.zeros(vecsize) for i in range(len(vec),maxsenlen)]
    return np.asarray(vec)

In [0]:
train_vectors = np.zeros((train_l,maxsenlen,vecsize))
for i in range(train_l):
    train_vectors[i,:,:] = load_vectors(Xtrain[i])

In [0]:
val_vectors = np.zeros((val_l,maxsenlen,vecsize))
for i in range(val_l):
    val_vectors[i,:,:] = load_vectors(Xval[i])

In [0]:
test_vectors = np.zeros((test_l,maxsenlen,vecsize))
for i in range(test_l):
    test_vectors[i,:,:] = load_vectors(Xtest[i])

In [0]:
def model_lstm():
    inp = Input(shape=(maxsenlen,vecsize))
    lstm1 = LSTM(100,recurrent_dropout=0.2,return_sequences=True)(inp)
    lstm1 = LSTM(50,return_sequences=False)(lstm1)
    out = Dense(1,activation='sigmoid')(lstm1)
    
    m1 = Model(inputs=inp,outputs=out)
    return m1

In [0]:
runs = 5
times= []
avg_acc= 0.0
avg_f1 = 0.0
for run in range(runs):
    model1 = model_lstm()
    if(run==0):
        print(model1.summary())
    model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    starttime = time.time()
    stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min')
    check = ModelCheckpoint('temp.h5' ,monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min', period=1)
    model1.fit(train_vectors,ytrain,validation_data=(val_vectors,yval),callbacks=[stop,check],batch_size=batchsize,
              epochs=numepochs,verbose=0)
    times.append(time.time()-starttime)
    ypred = model1.predict(test_vectors,128)
    ypred = (ypred>0.5)
    avg_f1+=f1_score(ypred,ytest)
    avg_acc+=accuracy_score(ypred,ytest)
    
print("F1 Score {} ".format(avg_f1/runs))
print("Accuracy Score {} ".format(avg_acc/runs))
print("Avg_time  {}".format(np.mean(np.asarray(times))))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_33 (InputLayer)        (None, 30, 100)           0         
_________________________________________________________________
lstm_29 (LSTM)               (None, 30, 100)           80400     
_________________________________________________________________
lstm_30 (LSTM)               (None, 50)                30200     
_________________________________________________________________
dense_63 (Dense)             (None, 1)                 51        
Total params: 110,651
Trainable params: 110,651
Non-trainable params: 0
_________________________________________________________________
None
F1 Score 0.4942114231982675 
Accuracy Score 0.5365222284749578 
Avg_time  362.170140171051


## PV(DBOW)+FNN

In [0]:
tagged_documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(words)]

In [0]:
np.random.seed(0)
pv_dbow = Doc2Vec(vector_size=vecsize, window=2, min_count=1, workers=4)
pv_dbow.build_vocab(tagged_documents)
pv_dbow.train(tagged_documents,total_examples=pv_dbow.corpus_count,epochs=20)

In [0]:
train_vectors = np.zeros((train_l,vecsize))
for i in range(train_l):
  train_vectors[i] = pv_dbow.infer_vector(Xtrain[i].split())

In [0]:
val_vectors = np.zeros((val_l,vecsize))
for i in range(val_l):
  val_vectors[i] = pv_dbow.infer_vector(Xval[i].split())

In [0]:
test_vectors = np.zeros((test_l,vecsize))
for i in range(test_l):
  test_vectors[i] = pv_dbow.infer_vector(Xtest[i].split())

In [0]:
def model_dense():
    inp = Input(shape=(vecsize,))
    dense1 = Dense(50,activation='relu')(inp)
    dense1 = Dropout(0.5)(dense1)
    dense1 = Dense(16,activation='relu')(dense1)
    dense1 = Dropout(0.5)(dense1)
    out = Dense(1,activation='sigmoid')(dense1)
    m1 = Model(inputs=inp,outputs=out)
    return m1

In [0]:
np.random.seed(0)
runs = 5
times= []
avg_acc= 0.0
avg_f1 = 0.0
for run in range(runs):
    model2 = model_dense()
    if(run==0):
        print(model2.summary())
    model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    starttime = time.time()
    stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min')
    check = ModelCheckpoint('temp.h5' ,monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min', period=1)
    model2.fit(train_vectors,ytrain,validation_data=(val_vectors,yval),callbacks=[stop,check],batch_size=batchsize,
              epochs=numepochs,verbose=0)
    times.append(time.time()-starttime)
    ypred = model2.predict(test_vectors,128)
    ypred = (ypred>0.5)
    avg_f1+=f1_score(ypred,ytest)
    avg_acc+=accuracy_score(ypred,ytest)
    
print("F1 Score {} ".format(avg_f1/runs))
print("Accuracy Score {} ".format(avg_acc/runs))
print("Avg_time  {}".format(np.mean(np.asarray(times))))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_38 (InputLayer)        (None, 100)               0         
_________________________________________________________________
dense_68 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_38 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_69 (Dense)             (None, 16)                816       
_________________________________________________________________
dropout_39 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_70 (Dense)             (None, 1)                 17        
Total params: 5,883
Trainable params: 5,883
Non-trainable params: 0
_________________________________________________________________
None
F1 

### PV(DM)+FNN

In [0]:
np.random.seed(0)
pv_dm = Doc2Vec(dm=1,vector_size=vecsize, window=2, min_count=1, workers=4)
pv_dm.build_vocab(tagged_documents)
pv_dm.train(tagged_documents,total_examples=pv_dm.corpus_count,epochs=20)

In [0]:
train_vectors = np.zeros((train_l,vecsize))
for i in range(train_l):
  train_vectors[i] = pv_dm.infer_vector(Xtrain[i].split())

In [0]:
val_vectors = np.zeros((val_l,vecsize))
for i in range(val_l):
  val_vectors[i] = pv_dm.infer_vector(Xval[i].split())

In [0]:
test_vectors = np.zeros((test_l,vecsize))
for i in range(test_l):
  test_vectors[i] = pv_dm.infer_vector(Xtest[i].split())

In [0]:
np.random.seed(0)
runs = 5
times= []
avg_acc= 0.0
avg_f1 = 0.0
for run in range(runs):
    model2 = model_dense()
    if(run==0):
        print(model2.summary())
    model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    starttime = time.time()
    stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min')
    check = ModelCheckpoint('temp.h5' ,monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min', period=1)
    model2.fit(train_vectors,ytrain,validation_data=(val_vectors,yval),callbacks=[stop,check],batch_size=batchsize,
              epochs=numepochs,verbose=0)
    times.append(time.time()-starttime)
    ypred = model2.predict(test_vectors,128)
    ypred = (ypred>0.5)
    avg_f1+=f1_score(ypred,ytest)
    avg_acc+=accuracy_score(ypred,ytest)
    
print("F1 Score {} ".format(avg_f1/runs))
print("Accuracy Score {} ".format(avg_acc/runs))
print("Avg_time  {}".format(np.mean(np.asarray(times))))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_43 (InputLayer)        (None, 100)               0         
_________________________________________________________________
dense_83 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_48 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_84 (Dense)             (None, 16)                816       
_________________________________________________________________
dropout_49 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_85 (Dense)             (None, 1)                 17        
Total params: 5,883
Trainable params: 5,883
Non-trainable params: 0
_________________________________________________________________
None
F1 

In [0]:
embedding_index = {}
with open('glove.840B.300d.txt','r',encoding='utf-8') as f:
  for line in f.readlines():
    wv = line.split(' ')
    word = wv[0]
    vector = np.asarray(wv[1:],'float32')
    embedding_index[word] = vector

In [0]:
maxsenlen = 30
embeddim = 300

In [0]:
tok = Tokenizer()
tok.fit_on_texts(Xtrain)

word_index = tok.word_index

train = tok.texts_to_sequences(Xtrain)
train_vectors = pad_sequences(train,maxsenlen)

val = tok.texts_to_sequences(Xval)
val_vectors = pad_sequences(val,maxsenlen)

test = tok.texts_to_sequences(Xtest)
test_vectors = pad_sequences(test,maxsenlen)

In [0]:
embedding_matrix = np.zeros((len(word_index)+1,embeddim))
for word, i in word_index.items():
  embedding_vector = embedding_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

### LSTM

In [0]:
def lstm_model():
  inp = Input(shape=(maxsenlen,))
  
  embed = Embedding(len(word_index)+1,embeddim,weights=[embedding_matrix],trainable=True)(inp)
  
  lstm_out = CuDNNLSTM(100,return_sequences=True)(embed)
  lstm_out = CuDNNLSTM(50,return_sequences=False)(lstm_out)
  
  out = Dense(16,activation='relu')(lstm_out)
  out = Dropout(0.5)(out)
  out = Dense(1,activation='sigmoid')(out)
  
  m2 = Model(inputs=inp,outputs=[out])
  return m2

In [61]:
np.random.seed(0)
runs = 5
times= []
avg_acc= 0.0
avg_f1 = 0.0
for run in range(runs):
    model3 = lstm_model()
    if(run==0):
        print(model3.summary())
    model3.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    starttime = time.time()
    stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min')
    check = ModelCheckpoint('temp.h5' ,monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min', period=1)
    model3.fit(train_vectors,ytrain,validation_data=(val_vectors,yval),callbacks=[stop,check],batch_size=batchsize,
              epochs=numepochs,verbose=0)
    times.append(time.time()-starttime)
    ypred = model3.predict(test_vectors,128)
    ypred = (ypred>0.5)
    avg_f1+=f1_score(ypred,ytest)
    avg_acc+=accuracy_score(ypred,ytest)
    
print("F1 Score {} ".format(avg_f1/runs))
print("Accuracy Score {} ".format(avg_acc/runs))
print("Avg_time  {}".format(np.mean(np.asarray(times))))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_110 (InputLayer)       (None, 30)                0         
_________________________________________________________________
embedding_110 (Embedding)    (None, 30, 300)           4176600   
_________________________________________________________________
cu_dnnlstm_213 (CuDNNLSTM)   (None, 30, 100)           160800    
_________________________________________________________________
cu_dnnlstm_214 (CuDNNLSTM)   (None, 50)                30400     
_________________________________________________________________
dense_213 (Dense)            (None, 16)                816       
_________________________________________________________________
dropout_102 (Dropout)        (None, 16)                0         
_________________________________________________________________
dense_214 (Dense)            (None, 1)                 17        
Total para

## Bi-LSTM

In [0]:
def bilstm_model():
  inp = Input(shape=(maxsenlen,))
  
  embed = Embedding(len(word_index)+1,embeddim,weights=[embedding_matrix],trainable=True)(inp)
  
  lstm_out = Bidirectional(CuDNNLSTM(100,return_sequences=True))(embed)
  lstm_out = Bidirectional(CuDNNLSTM(50,return_sequences=False))(lstm_out)
  
  out = Dense(16,activation='relu')(lstm_out)
  out = Dropout(0.5)(out)
  out = Dense(1,activation='sigmoid')(out)
  
  m3 = Model(inputs=inp,outputs=[out])
  return m3

In [60]:
np.random.seed(0)
runs = 5
times= []
avg_acc= 0.0
avg_f1 = 0.0
for run in range(runs):
    model4 = bilstm_model()
    if(run==0):
        print(model4.summary())
    model4.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    starttime = time.time()
    stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min')
    check = ModelCheckpoint('temp.h5' ,monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min', period=1)
    model4.fit(train_vectors,ytrain,validation_data=(val_vectors,yval),callbacks=[stop,check],batch_size=batchsize,
              epochs=numepochs,verbose=0)
    times.append(time.time()-starttime)
    ypred = model4.predict(test_vectors,128)
    ypred = (ypred>0.5)
    avg_f1+=f1_score(ypred,ytest)
    avg_acc+=accuracy_score(ypred,ytest)
    
print("F1 Score {} ".format(avg_f1/runs))
print("Accuracy Score {} ".format(avg_acc/runs))
print("Avg_time  {}".format(np.mean(np.asarray(times))))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_105 (InputLayer)       (None, 30)                0         
_________________________________________________________________
embedding_105 (Embedding)    (None, 30, 300)           4176600   
_________________________________________________________________
bidirectional_79 (Bidirectio (None, 30, 200)           321600    
_________________________________________________________________
bidirectional_80 (Bidirectio (None, 100)               100800    
_________________________________________________________________
dense_203 (Dense)            (None, 16)                1616      
_________________________________________________________________
dropout_97 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_204 (Dense)            (None, 1)                 17        
Total para

In [0]:
num_filters = 100
filter_sizes = [3,4,5]
drop = 0.5

## Random CNN

In [0]:
def random_cnn():
  inp = Input(shape=(maxsenlen,), dtype='int32')
  embedding = Embedding(input_dim=len(word_index)+1, output_dim=embeddim, input_length=maxsenlen)(inp)
  reshape = Reshape((maxsenlen,embeddim,1))(embedding)

  conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embeddim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
  conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embeddim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
  conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embeddim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)

  maxpool_0 = MaxPool2D(pool_size=(maxsenlen - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
  maxpool_1 = MaxPool2D(pool_size=(maxsenlen - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
  maxpool_2 = MaxPool2D(pool_size=(maxsenlen - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)

  concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
  flatten = Flatten()(concatenated_tensor)
  dropout = Dropout(drop)(flatten)
  output = Dense(units=1, activation='sigmoid')(dropout)
  
  m4 = Model(inputs=inp,outputs=output)
  return m4

In [0]:
np.random.seed(0)
runs = 5
times= []
avg_acc= 0.0
avg_f1 = 0.0
for run in range(runs):
    model5 = random_cnn()
    if(run==0):
        print(model5.summary())
    model5.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    starttime = time.time()
    stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min')
    check = ModelCheckpoint('temp.h5' ,monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min', period=1)
    model5.fit(train_vectors,ytrain,validation_data=(val_vectors,yval),callbacks=[stop,check],batch_size=batchsize,
              epochs=numepochs,verbose=0)
    times.append(time.time()-starttime)
    ypred = model5.predict(test_vectors,128)
    ypred = (ypred>0.5)
    avg_f1+=f1_score(ypred,ytest)
    avg_acc+=accuracy_score(ypred,ytest)
    
print("F1 Score {} ".format(avg_f1/runs))
print("Accuracy Score {} ".format(avg_acc/runs))
print("Avg_time  {}".format(np.mean(np.asarray(times))))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_59 (InputLayer)           (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_27 (Embedding)        (None, 30, 300)      4176600     input_59[0][0]                   
__________________________________________________________________________________________________
reshape_8 (Reshape)             (None, 30, 300, 1)   0           embedding_27[0][0]               
__________________________________________________________________________________________________
conv2d_22 (Conv2D)              (None, 28, 1, 100)   90100       reshape_8[0][0]                  
__________________________________________________________________________________________________
conv2d_23 

### Static CNN

In [0]:
def static_cnn():
  inp = Input(shape=(maxsenlen,), dtype='int32')
  embedding = Embedding(input_dim=len(word_index)+1, output_dim=embeddim, weights=[embedding_matrix],trainable=False)(inp)
  reshape = Reshape((maxsenlen,embedddim,1))(embedding)

  conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embeddim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
  conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embeddim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
  conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embeddim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)

  maxpool_0 = MaxPool2D(pool_size=(maxsenlen - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
  maxpool_1 = MaxPool2D(pool_size=(maxsenlen - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
  maxpool_2 = MaxPool2D(pool_size=(maxsenlen - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)

  concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
  flatten = Flatten()(concatenated_tensor)
  dropout = Dropout(drop)(flatten)
  output = Dense(units=1, activation='sigmoid')(dropout)
  
  m5 = Model(inputs=inp,outputs=output)
  return m5

In [0]:
np.random.seed(0)
runs = 5
times= []
avg_acc= 0.0
avg_f1 = 0.0
for run in range(runs):
    model6 = random_cnn()
    if(run==0):
        print(model6.summary())
    model6.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    starttime = time.time()
    stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min')
    check = ModelCheckpoint('temp.h5' ,monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min', period=1)
    model6.fit(train_vectors,ytrain,validation_data=(val_vectors,yval),callbacks=[stop,check],batch_size=batchsize,
              epochs=numepochs,verbose=0)
    times.append(time.time()-starttime)
    ypred = model6.predict(test_vectors,128)
    ypred = (ypred>0.5)
    avg_f1+=f1_score(ypred,ytest)
    avg_acc+=accuracy_score(ypred,ytest)
    
print("F1 Score {} ".format(avg_f1/runs))
print("Accuracy Score {} ".format(avg_acc/runs))
print("Avg_time  {}".format(np.mean(np.asarray(times))))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_64 (InputLayer)           (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_32 (Embedding)        (None, 30, 300)      4176600     input_64[0][0]                   
__________________________________________________________________________________________________
reshape_13 (Reshape)            (None, 30, 300, 1)   0           embedding_32[0][0]               
__________________________________________________________________________________________________
conv2d_37 (Conv2D)              (None, 28, 1, 100)   90100       reshape_13[0][0]                 
__________________________________________________________________________________________________
conv2d_38 